In [8]:
# Passo 1: Instale as bibliotecas necessárias
!pip install openai python-dotenv semantic-kernel --quiet

# Passo 2: Importe as bibliotecas
import os
from openai import AzureOpenAI
import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.core_plugins.text_plugin import TextPlugin
from semantic_kernel.functions.kernel_function_decorator import kernel_function

# Passo 3: Defina suas variáveis de ambiente
os.environ["AZURE_OPENAI_ENDPOINT"] = "https://eitasenhor.openai.azure.com/"
os.environ["AZURE_OPENAI_API_KEY"] = "---"
os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"] = "gpt-35-turbo"  # Nome do modelo implantado no Azure

# Passo 4: Crie o cliente da API do Azure OpenAI
client = AzureOpenAI(
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    api_version="2024-02-01"
)

print("Fazendo chamada direta à API do Azure OpenAI...\n")

try:
    response = client.chat.completions.create(
        model=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME"),
        messages=[
            {"role": "user", "content": "Explique o conceito de sistemas distribuídos."}
        ],
        temperature=0.7,
        max_tokens=200
    )

    print("✅ Resposta da API do Azure OpenAI:")
    print(response.choices[0].message.content)
except Exception as e:
    print(f"❌ Erro ao chamar a API: {e}")

# Passo 5: Inicialize o Semantic Kernel
print("\nIniciando o Semantic Kernel...\n")

try:
    kernel = sk.Kernel()

    # Adiciona o serviço de chat usando o Azure OpenAI
    kernel.add_service(
        service=AzureChatCompletion(
            deployment_name=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME"),
            endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
            api_key=os.getenv("AZURE_OPENAI_API_KEY")
        )
    )

    # Plugin de texto básico (exemplo)
    text_plugin = TextPlugin()
    kernel.add_plugin(plugin=text_plugin, plugin_name="text")

    # Função semântica simples (cria uma função inline)
    prompt = """
    Responda à seguinte pergunta:
    {{ $input }}
    """

    summarize_function = kernel.add_function(
        function_name="summarize",
        plugin_name="text",
        prompt=prompt,
        description="Responde a perguntas fornecidas pelo usuário.",
        input_description="Pergunta do usuário."
    )

    # Execute a função com um input real
    result = await kernel.invoke(summarize_function, input="O que é topologia de rede em estrela?")

    print("✅ Resultado do Semantic Kernel:")
    print(result)

except Exception as e:
    print(f"❌ Erro ao usar Semantic Kernel: {e}")

Fazendo chamada direta à API do Azure OpenAI...

✅ Resposta da API do Azure OpenAI:
Sistemas distribuídos referem-se a um conjunto de computadores interconectados que trabalham juntos para alcançar um objetivo comum. Esses computadores estão localizados em diferentes locais físicos e se comunicam através de uma rede de computadores. Em um sistema distribuído, cada computador tem sua própria memória e processador, e pode operar de forma independente, mas ao mesmo tempo colaborar com os outros computadores para realizar tarefas complexas.

Esses sistemas são projetados para melhorar o desempenho, escalabilidade e disponibilidade de serviços e aplicações, distribuindo a carga de trabalho entre os computadores da rede. Eles também oferecem maior tolerância a falhas, uma vez que, se um dos computadores falhar, os outros podem continuar operando sem interrupções.

Os sistemas distribuídos são amplamente utilizados em diferentes áreas, como redes sociais

Iniciando o Semantic Kernel...

✅ Res